
## Import Packages

In [ ]:
import zipfile as zf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import cv2
import os
from tqdm.notebook import tqdm
from PIL import Image

import future

from torch.utils.tensorboard import SummaryWriter

%load_ext autoreload
%autoreload 2

## Define the ENet model

We decided to model following residual blocks as separate class to model ENET encoder and decoder:
    - Initial block
    - RDDNeck - class for regular, downsampling and dilated bottlenecks
    - ASNeck - class for asymetric bottlenecks
    - UBNeck - class for upsampling bottlenecks

ENET architecture is autoencoder based model and is divided into 5 sub-blocks. Pleas refer [ENET paper](https://arxiv.org/pdf/1606.02147.pdf) for details of each sub-block. ENET building blocks code is taken from [here](https://github.com/iArunava/ENet-Real-Time-Semantic-Segmentation).

Fast scene understanding uses first 2 sub-blocks as encoder and remaining 3 as decoder. In this implemantation, there is 1 shared encoder and 3 separate decoder for 3 tasks(instance segementation, semantic segmentation, Depth estimation )

In [ ]:
import os, sys
nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
print(nb_dir)

In [ ]:
from models.ENetDecoder import ENetDecoder
from models.ENetEncoder import ENetEncoder

class BranchedENet(nn.Module):
    def __init__(self, C):
        super().__init__()
        # Define class variables
        # C - number of classes
        self.C = C
        self.enc = ENetEncoder(C)
        self.dec1 = ENetDecoder(C)
        self.dec2 = ENetDecoder(C)
        self.dec3 = ENetDecoder(C)
        
        
    def forward(self, x):
        # Output of Encoder
        x, i1, i2 = self.enc(x)
        # output of all 3 decoder in list
        #x = torch.stack([self.dec1(x, i1, i2), self.dec2(x, i1, i2), self.dec3(x, i1, i2)])
        x = (self.dec1(x, i1, i2), self.dec2(x, i1, i2), self.dec3(x, i1, i2))
        return x

## Instantiate the ENet model

In [ ]:
enet = BranchedENet(35)

In [ ]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
enet = enet.to(device)

## Define Dataloader

In [ ]:
from data.cityscapes import Cityscapes as dataset

In [ ]:
height = 512
width = 1024
dataset_dir = 'data/cityscape'
image_transform = transforms.Compose(
        [transforms.Resize((height,width)),transforms.ToTensor()])
train_set = dataset(dataset_dir,transform=image_transform)

train_loader = data.DataLoader(train_set,batch_size=1,shuffle=True,
        num_workers=1)

## 3 - Losses(todo)
(1) Semantic Segmentation Loss

(2) Instantance Segmentation Loss

(3) Depth Estimation Loss

In [ ]:
"""
This is the implementation of following paper:
https://arxiv.org/pdf/1802.05591.pdf
This implementation is based on following code:
https://github.com/Wizaron/instance-segmentation-pytorch
"""
from torch.nn.modules.loss import _Loss
from torch.autograd import Variable
import torch


class DiscriminativeLoss(_Loss):

    def __init__(self, delta_var=0.5, delta_dist=1.5,
                 norm=2, alpha=1.0, beta=1.0, gamma=0.001,
                 usegpu=True, size_average=True):
        super(DiscriminativeLoss, self).__init__(size_average)
        self.delta_var = delta_var
        self.delta_dist = delta_dist
        self.norm = norm
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.usegpu = usegpu
        assert self.norm in [1, 2]

    def forward(self, input, target, n_clusters):
#         _assert_no_grad(target)
        return self._discriminative_loss(input, target, n_clusters)

    def _discriminative_loss(self, input, target, n_clusters):
        bs, n_features, height, width = input.size()
        max_n_clusters = target.size(1)

        input = input.contiguous().view(bs, n_features, height * width)
        target = target.contiguous().view(bs, max_n_clusters, height * width)

        c_means = self._cluster_means(input, target, n_clusters)
        l_var = self._variance_term(input, target, c_means, n_clusters)
        l_dist = self._distance_term(c_means, n_clusters)
        l_reg = self._regularization_term(c_means, n_clusters)

        loss = self.alpha * l_var + self.beta * l_dist + self.gamma * l_reg

        return loss

    def _cluster_means(self, input, target, n_clusters):
        bs, n_features, n_loc = input.size()
        max_n_clusters = target.size(1)

        # bs, n_features, max_n_clusters, n_loc
        input = input.unsqueeze(2).expand(bs, n_features, max_n_clusters, n_loc)
        # bs, 1, max_n_clusters, n_loc
        target = target.unsqueeze(1)
        # bs, n_features, max_n_clusters, n_loc
        input = input * target

        means = []
        for i in range(bs):
            # n_features, n_clusters, n_loc
            input_sample = input[i, :, :n_clusters[i]]
            # 1, n_clusters, n_loc,
            target_sample = target[i, :, :n_clusters[i]]
            # n_features, n_cluster
            mean_sample = input_sample.sum(2) / (target_sample.sum(2) + 0.00001)

            # padding
            n_pad_clusters = max_n_clusters - n_clusters[i]
            assert n_pad_clusters >= 0
            if n_pad_clusters > 0:
                pad_sample = torch.zeros(n_features, n_pad_clusters)
                pad_sample = Variable(pad_sample)
                if self.usegpu:
                    pad_sample = pad_sample.cuda()
                mean_sample = torch.cat((mean_sample, pad_sample), dim=1)
            means.append(mean_sample)

        # bs, n_features, max_n_clusters
        means = torch.stack(means)

        return means

    def _variance_term(self, input, target, c_means, n_clusters):
        bs, n_features, n_loc = input.size()
        max_n_clusters = target.size(1)

        # bs, n_features, max_n_clusters, n_loc
        c_means = c_means.unsqueeze(3).expand(bs, n_features, max_n_clusters, n_loc)
        # bs, n_features, max_n_clusters, n_loc
        input = input.unsqueeze(2).expand(bs, n_features, max_n_clusters, n_loc)
        # bs, max_n_clusters, n_loc
        var = (torch.clamp(torch.norm((input - c_means), self.norm, 1) -
                           self.delta_var, min=0) ** 2) * target

        var_term = 0
        for i in range(bs):
            # n_clusters, n_loc
            var_sample = var[i, :n_clusters[i]]
            # n_clusters, n_loc
            target_sample = target[i, :n_clusters[i]]

            # n_clusters
            c_var = var_sample.sum(1) / (target_sample.sum(1) + 0.00001)
            var_term += c_var.sum() / int(n_clusters[i])
        var_term /= bs

        return var_term

    def _distance_term(self, c_means, n_clusters):
        bs, n_features, max_n_clusters = c_means.size()

        dist_term = 0
        for i in range(bs):
            if n_clusters[i] <= 1:
                continue

            # n_features, n_clusters
            mean_sample = c_means[i, :, :n_clusters[i]]

            # n_features, n_clusters, n_clusters
            means_a = mean_sample.unsqueeze(2).expand(n_features, n_clusters[i], n_clusters[i])
            means_b = means_a.permute(0, 2, 1)
            diff = means_a - means_b

            margin = 2 * self.delta_dist * (1.0 - torch.eye(n_clusters[i]))
            margin = Variable(margin)
            if self.usegpu:
                margin = margin.cuda()
            c_dist = torch.sum(torch.clamp(margin - torch.norm(diff, self.norm, 0), min=0) ** 2)
            dist_term += c_dist / (2 * n_clusters[i] * (n_clusters[i] - 1))
        dist_term /= bs

        return dist_term

    def _regularization_term(self, c_means, n_clusters):
        bs, n_features, max_n_clusters = c_means.size()

        reg_term = 0
        for i in range(bs):
            # n_features, n_clusters
            mean_sample = c_means[i, :, :n_clusters[i]]
            reg_term += torch.mean(torch.norm(mean_sample, self.norm, 0))
        reg_term /= bs

        return reg_term

In [ ]:
def inverse_huber_loss(out, target):
    absdiff = torch.abs(out-target)
    C = 0.2*torch.max(absdiff)
    return torch.mean(torch.where(absdiff<C, absdiff, (absdiff*absdiff+C*C)/(2*C)))

In [ ]:
# local grad = require 'autograd'

# def lossfunction(lossf_name, weights):
#     if (lossf_name == 'softmaxLoss') then
#         lossfunction = cudnn.SpatialCrossEntropyCriterion(weights)
#     elseif (lossf_name == 'huberLoss') then
#         lossfunction = grad.nn.AutoCriterion('depthLoss_huber')(require 'lossf/myHuberLoss')
#     elseif (lossf_name == 'instanceLoss') then
#         lossfunction = grad.nn.AutoCriterion('instance_loss')(require 'lossf/myInstanceLoss')
#     else
#         assert(false, 'Cannot load lossfunction ' .. opts.lossf)
#     end

#     return lossfunction
# end

# return getLoss

# Step 5 and 6 has been done in dataloader


## 7 - Define the Hyperparameters(todo)

In [ ]:
from data.utils import enet_weighing
lr = 5e-4
batch_size = 1

# figure out enet_weighing issue
#criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(enet_weighing(train_loader, 12)).to(device))
criterion = nn.CrossEntropyLoss().to(device)
criterion_dpth = torch.nn.MSELoss(reduction='mean').to(device)
'''
criterion_disc = DiscriminativeLoss(delta_var=0.1,
                                       delta_dist=0.6,
                                       norm=2,
                                       usegpu=True).to(device)
'''
optimizer = torch.optim.Adam(enet.parameters(), lr=lr, weight_decay=2e-4)

## Name experiment to differentiate different runs for tensorboard
## eg. for hyperparameter tuning
experiment = 'experiment_lr-' + str(lr)

print_every = 5
eval_every = 2

## 8 - Training loop(todo)

In [ ]:
train_losses = []
eval_losses = []
bc_train = 367 // batch_size # mini_batch train
bc_eval = 101 // batch_size  # mini_batch validation
epochs = 100
train_writer = SummaryWriter('runs/' + experiment + '/train')
val_writer = SummaryWriter('runs/' + experiment + '/val')

In [ ]:
# Train loop

for e in range(1, epochs+1):     
    train_loss = 0
    print ('-'*15,'Epoch %d' % e, '-'*15)
    enet.train()
    for _ in tqdm(range(bc_train)):
        img, label, inst, dpth = dataiter.next()

        # assign data to cpu/gpu
        img, label, inst, dpth = img.to(device), label.to(device), inst.to(device), dpth.to(device)
        label = label.squeeze(1)
        inst = inst.squeeze(1)
        dpth = dpth.squeeze(1)
        
        # set non-car labels to 0 for inst
#         inst[inst!=26] = 0
#         #inst[inst!=15] = 0
    
        optimizer.zero_grad()
        out = enet(img.float())

        # split output into three predictions
        label_out, inst_out, dpth_out = out[0], out[1], out[2]
    
        # get pixel-wise sum for depth
        dpth_out = torch.sum(dpth_out, dim=1)

        # loss calculation for class segmentation
        loss = criterion(label_out, label.long()).float()

        # loss calculation for class instance
        loss += criterion(inst_out, inst.long()).float()
        
        #[5] in the below line indicates no. of clusters or no. of instances. Should be taken from the image
        #loss += criterion_disc(inst_out, inst.long(), [5] * len(inst)).float()

        # loss calculation for depth
        loss += criterion_dpth(dpth_out, dpth.float())
        loss.backward()
        
        # update weights
        optimizer.step()

        train_loss += loss.item()
        
    train_writer.add_scalar('Loss', train_loss, e)
    
    if e % eval_every == 0:
        with torch.no_grad():
            enet.eval()
            
            eval_loss = 0

            # Validation loop
            for _ in tqdm(range(bc_eval)):
                img, label, inst, dpth = dataiter.next()
                img, label, inst, dpth = img.to(device), label.to(device), inst.to(device), dpth.to(device)
                label = label.squeeze(1)
                inst = inst.squeeze(1)
                dpth = dpth.squeeze(1)
        
                out = enet(img.float())
                
                # split output into three predictions
                label_out, inst_out, dpth_out = out[0,:,:,:,:], out[1,:,:,:,:], out[2,:,:,:,:]

                # get pixel-wise sum for depth
                dpth_out = torch.sum(dpth_out, dim=1)

                # loss calculation for class segmentation
                eval_loss += criterion(label_out, label.long()).float().item()

                # loss calculation for class instance
                eval_loss += criterion(inst_out, inst.long()).float().item()
                #eval_loss += criterion_disc(inst_out, inst.long(), [5] * len(inst)).float().item()

                # loss calculation for depth
                eval_loss += criterion_dpth(dpth_out, dpth.float()).item()
                
            
            val_writer.add_scalar('Loss', eval_loss, e)
        
    if e % print_every == 0:
        checkpoint = {
            'epochs' : e,
            'state_dict' : enet.state_dict()
        }
        torch.save(checkpoint, '/content/ckpt-enet-{}-{}.pth'.format(e, train_loss))
        print ('Model saved!')
    train_writer.flush()
    val_writer.flush()

print ('Epoch {}/{}...'.format(e, epochs),
       'Total Mean Loss: {:6f}'.format(sum(train_losses) / epochs))

train_writer.close()
val_writer.close()

In [ ]:
import os

In [ ]:
for path, _, files in os.walk(folder):

In [ ]:
import torch
def discriminative_loss_single(prediction, correct_label, feature_dim, label_shape, 
                            delta_v, delta_d, param_var, param_dist, param_reg):
    
    ''' Discriminative loss for a single prediction/label pair.
    :param prediction: inference of network
    :param correct_label: instance label
    :feature_dim: feature dimension of prediction
    :param label_shape: shape of label
    :param delta_v: cutoff variance distance
    :param delta_d: curoff cluster distance
    :param param_var: weight for intra cluster variance
    :param param_dist: weight for inter cluster distances
    :param param_reg: weight regularization
    '''

    ### Reshape so pixels are aligned along a vector
    correct_label = correct_label.reshape([label_shape[1]*label_shape[0]])
    print('correct_label')
    print(correct_label)
    print('\n')
    reshaped_pred = prediction.reshape([label_shape[1]*label_shape[0], feature_dim])
    print('reshaped_pred')
    print(reshaped_pred)
    print('\n')
    ### Count instances
    unique_labels, unique_id, counts = correct_label.unique(sorted=False, return_inverse=True, return_counts=True)
    #counts = torch.stack([(correct_label==x_u).sum() for x_u in unique_labels])
    print('unique_labels')
    print(unique_labels)
    print('\n')
    print('unique_id')
    print(unique_id)
    print('\n')
    print('counts')
    print(counts)
    print('\n')
    counts = counts.float()
    print('counts')
    print(counts)
    print('\n')
    num_instances = torch.tensor(torch.numel(unique_labels))
    print('num_instances')
    print(num_instances)
    print('\n')
    
    
    segmented_sum = torch.zeros_like(reshaped_pred).scatter_add_(1, unique_id, reshaped_pred)
    
    
    mu = torch.floor(torch.div(segmented_sum, torch.reshape(counts, (-1, 1))))
    mu_expand = torch.gather(mu, unique_id)

    ### Calculate l_var
    distance = torch.norm(tf.subtract(mu_expand, reshaped_pred), dim=1)
    distance = torch.sub(distance, delta_v)
    distance = torch.clamp(distance, min=0., max=distance)
    distance = torch.mul(distance, distance)

    l_var = tf.unsorted_segment_sum(distance, unique_id, num_instances)
    l_var = torch.floor(torch.div(l_var, counts))
    l_var = l_var.sum()
    l_var = torch.divide(l_var, num_instances.float())
    
    ### Calculate l_dist
    
    # Get distance for each pair of clusters like this:
    #   mu_1 - mu_1
    #   mu_2 - mu_1
    #   mu_3 - mu_1
    #   mu_1 - mu_2
    #   mu_2 - mu_2
    #   mu_3 - mu_2
    #   mu_1 - mu_3
    #   mu_2 - mu_3
    #   mu_3 - mu_3

    mu_interleaved_rep = tf.tile(mu, [num_instances, 1])
    mu_band_rep = tf.tile(mu, [1, num_instances])
    mu_band_rep = mu_band_rep.reshape([num_instances*num_instances, feature_dim])

    mu_diff = torch.sub(mu_band_rep, mu_interleaved_rep)
    
    # Filter out zeros from same cluster subtraction
    intermediate_tensor = torch.sum(tf.abs(mu_diff),axis=1)
    zero_vector = torch.zeros(1, dtype=tf.float32)
    bool_mask = torch.ne(intermediate_tensor, zero_vector)
    mu_diff_bool = tf.torch.masked_select(mu_diff, bool_mask)

    mu_norm = torch.norm(mu_diff_bool, p=2, axis=1)
    mu_norm = torch.sub(2.*delta_d, mu_norm)
    mu_norm = torch.clamp(mu_norm, min=0., max=mu_norm)
    mu_norm = torch.mul(mu_norm,mu_norm)

    l_dist = torch.mean(mu_norm)

    ### Calculate l_reg
    l_reg = torch.mean(torch.norm(mu, p=2, dim=1))

    param_scale = 1.
    l_var = param_var * l_var
    l_dist = param_dist * l_dist
    l_reg = param_reg * l_reg

    loss = param_scale*(l_var + l_dist + l_reg)
      
    return True
    


def discriminative_loss(prediction, correct_label, feature_dim, image_shape, 
                delta_v, delta_d, param_var, param_dist, param_reg):
    ''' Iterate over a batch of prediction/label and cumulate loss
    :return: discriminative loss and its three components
    '''
    def cond(label, batch, out_loss, out_var, out_dist, out_reg, i):
        return tf.less(i, tf.shape(batch)[0])

    def body(label, batch, out_loss, out_var, out_dist, out_reg, i):
        disc_loss, l_var, l_dist, l_reg = discriminative_loss_single(prediction[i], correct_label[i], feature_dim, image_shape, 
                        delta_v, delta_d, param_var, param_dist, param_reg)

        out_loss = out_loss.write(i, disc_loss)
        out_var = out_var.write(i, l_var)
        out_dist = out_dist.write(i, l_dist)
        out_reg = out_reg.write(i, l_reg)

        return label, batch, out_loss, out_var, out_dist, out_reg, i + 1

    # TensorArray is a data structure that support dynamic writing
    output_ta_loss = tf.TensorArray(dtype=tf.float32,
                   size=0,
                   dynamic_size=True)
    output_ta_var = tf.TensorArray(dtype=tf.float32,
                   size=0,
                   dynamic_size=True)
    output_ta_dist = tf.TensorArray(dtype=tf.float32,
                   size=0,
                   dynamic_size=True)
    output_ta_reg = tf.TensorArray(dtype=tf.float32,
                   size=0,
                   dynamic_size=True)

    _, _, out_loss_op, out_var_op, out_dist_op, out_reg_op, _  = tf.while_loop(cond, body, [correct_label, 
                                                        prediction, 
                                                        output_ta_loss, 
                                                        output_ta_var, 
                                                        output_ta_dist, 
                                                        output_ta_reg, 
                                                        0])
    out_loss_op = out_loss_op.stack()
    out_var_op = out_var_op.stack()
    out_dist_op = out_dist_op.stack()
    out_reg_op = out_reg_op.stack()
    
    disc_loss = tf.reduce_mean(out_loss_op)
    l_var = tf.reduce_mean(out_var_op)
    l_dist = tf.reduce_mean(out_dist_op)
    l_reg = tf.reduce_mean(out_reg_op)

    return disc_loss, l_var, l_dist, l_reg

In [ ]:
prediction = torch.tensor([[1,2061],[2,2062]])
correct_label = torch.tensor([[1,2061],[2,2062]])
disc_loss, l_var, l_dist, l_reg = discriminative_loss_single(prediction, correct_label, 1, (2,2), 
                            1, 1, 0.001, 0.5, 1.5)

In [ ]:
!nvidia-smi